In [1]:
using Laplacians

objc[34115]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[34115]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[34115]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[34115]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_7/"

"../graphs/se_7/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [10]:
@time F = treeSolver(tree)
@time F(b);

  0.003216 seconds (122 allocations: 2.858 MB)
  0.016836 seconds (157.47 k allocations: 3.014 MB, 41.46% gc time)


In [13]:
# a run on standard Float64 data types
numIts = 10000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 3.945417675864844e8
Working on iteration 20 with a-norm error 1.94025920172515e8
Working on iteration 30 with a-norm error 1.3108235925885397e8
Working on iteration 40 with a-norm error 9.319177763886963e7
Working on iteration 50 with a-norm error 7.12175886370184e7
Working on iteration 60 with a-norm error 5.798411934807251e7
Working on iteration 70 with a-norm error 4.9746433473172136e7
Working on iteration 80 with a-norm error 4.330523201388377e7
Working on iteration 90 with a-norm error 3.815280718315145e7
Working on iteration 100 with a-norm error 3.388398881882901e7
Working on iteration 110 with a-norm error 3.0839993383530192e7
Working on iteration 120 with a-norm error 2.8216074678706776e7
Working on iteration 130 with a-norm error 2.5969565322154623e7
Working on iteration 140 with a-norm error 2.425656375416882e7
Working on iteration 150 with a-norm error 2.2591492174413823e7
Working on iteration 160 with a-norm error 2.125453583789089

In [14]:
numIts = 10000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 3.945417675864717e8
Working on iteration 20 with a-norm error 1.9402592017248738e8
Working on iteration 30 with a-norm error 1.3108235925880003e8
Working on iteration 40 with a-norm error 9.31917776387446e7
Working on iteration 50 with a-norm error 7.121755573230349e7
Working on iteration 60 with a-norm error 5.76063466327223e7
Working on iteration 70 with a-norm error 4.94779147290977e7
Working on iteration 80 with a-norm error 4.325358494524775e7
Working on iteration 90 with a-norm error 3.815280711632138e7
Working on iteration 100 with a-norm error 3.385740396226583e7
Working on iteration 110 with a-norm error 3.0647699322842084e7
Working on iteration 120 with a-norm error 2.805409608739852e7
Working on iteration 130 with a-norm error 2.5836940853420913e7
Working on iteration 140 with a-norm error 2.4106318152863428e7
Working on iteration 150 with a-norm error 2.2390192643187735e7
Working on iteration 160 with a-norm error 2.1054164221811414